In [246]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json
import seaborn as sns
import pickle

In [247]:
#1) Calculate the cost of the scrap by scrap type from PO
#2) Calculate the estimated cost of lost yield
#3) Calculate the estimated copper breach probability cost

In [248]:
constraints = '../data/1/constraints.json'
heats_prev = '../data/1/previous_heats_with_properties.json'
schedule = '../data/1/production_schedule.json'
scrap_inventory = '../data/1/scrap_inventory.json'
scrap_orders = '../data/1/scrap_orders.json'

with open(constraints) as json_file:
    constraints = json.load(json_file)
with open(heats_prev) as json_file:
    heats_prev = json.load(json_file)
with open(schedule) as json_file:
    schedule = json.load(json_file)
with open(scrap_inventory) as json_file:
    scrap_inventory = json.load(json_file)
with open(scrap_orders) as json_file:
    scrap_orders = json.load(json_file)

In [249]:
scrap_orders = pd.DataFrame(scrap_orders)
scrap_orders['total_cost'] = scrap_orders['price_per_ton'] * scrap_orders['weight']
scrap_orders_grouped = scrap_orders.groupby('scrap_type').sum().reset_index()
scrap_orders_grouped['avg_price_per_ton'] = scrap_orders_grouped['total_cost']/scrap_orders_grouped['weight']
scrap_cost = dict(zip(scrap_orders_grouped.scrap_type, scrap_orders_grouped.avg_price_per_ton))

In [250]:
#2. Cost of lost yield
#Assume Hot Rolled Coil is 743.40/ton
#If yield is 100%, then you lose 0
#if yield is 90% then you lose 10%*total weight input * 743.40 

In [251]:
#3. If copper is breached, you lose say 50% of 743.40 per ton * required


In [252]:
def value_in_use(commodity_inputs, yield_model, copper_model, copper_target, scrap_cost):
    """
    value_in_use: calculate the value in use for a given set of commodity inputs
    
    inputs: 
        commodity_inputs - dictionary of scrap_type:scrap weight
        yield_model - unpickled yield model
        copper_model - unpickled copper model
        copper_target - production copper target
        scrap_cost - dictionary of scrap_type:scrap cost
    outputs:
        scrap_cost_total - the totaled scrap cost for the commodities
        yield_cost - the cost of lost yield
        copper_cost - the cost of copper (NEEDS WORK)
        value_in_use - the sum of all of these costs
    """
    total_inputs_weight = sum(commodity_inputs.values())
    commodity_inputs_normed = {}
    for c in commodities:
        commodity_inputs_normed[c] = commodity_inputs[c]/total_inputs_weight
    commodity_inputs_normed = pd.DataFrame(commodity_inputs_normed.values()).T
    
    yield_estimate = yield_model.predict(commodity_inputs_normed)
    yield_cost = (1-yield_estimate) * total_inputs_weight * 743.40
    
    copper_estimate = copper_model.predict(commodity_inputs_normed)
    print(copper_estimate)
    copper_cost = (copper_target - copper_estimate) * 743.40
    
    scrap_cost_total = 0
    for c in commodities:
        scrap_cost_total += commodity_inputs[c] * scrap_cost[c]
    value_in_use = yield_cost + copper_cost + scrap_cost_total
    return scrap_cost_total, yield_cost, copper_cost, value_in_use

In [254]:
commodity_inputs = {"bushling":300, "pig_iron":200, "municipal_shred":350, "skulls":200}
#Now we must figure out where the commodity inputs will come from

y_model = '../application/pickles/yield_model.pickle'
y_model = pickle.load(open(y_model, 'rb'))

cu_model = '../application/pickles/copper_model.pickle'
cu_model = pickle.load(open(cu_model, 'rb'))

value_in_use(commodity_inputs, yield_model, cu_model, 0.15, scrap_cost)

[0.09286499]


/Users/ajay.anand/anaconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


(86740.40404040404,
 array([79657.77832031]),
 array([42.47416626]),
 array([166440.65652698]))

In [242]:
commodities = ['bushling', 'pig_iron', 'municipal_shred', 'skulls']